In [1]:
# Core 
import datetime
import os
import glob
import tempfile
import shutil
import sys
import pickle
import warnings

# Analysis 
import xarray as xr
import numpy as np
import pandas as pd
import scipy as sp

# Plotting
import matplotlib.pyplot as plt

# Debugging 
import pdb, traceback
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint import animate


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
def CPOL_files_from_datetime_list(datetimes):
    print('Gathering files.')
    base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_150km_2500m/'
    filenames = []
    for i in range(len(datetimes)):
        year = str(datetimes[i])[0:4]
        month = str(datetimes[i])[5:7]
        day = str(datetimes[i])[8:10]
        hour = str(datetimes[i])[11:13]
        minute = str(datetimes[i])[14:16]
        filename = (base + '{0}/{0}{1}{2}/'.format(year, month, day) 
                    + 'CPOL_{0}{1}{2}'.format(year, month, day)
                    + '_{}{}_GRIDS_2500m.nc'.format(hour, minute))
        if os.path.isfile(filename):
            filenames.append(filename)
    
    return sorted(filenames), datetimes[0], datetimes[-1]
    
def CPOL_files_from_TINT_obj(tracks_obj, uid):
    datetimes = tracks_obj.system_tracks.xs(uid, level='uid')
    datetimes = datetimes.reset_index(level='time')['time']
    datetimes = list(datetimes.values)
    [files, start_date, end_date] = CPOL_files_from_datetime_list(datetimes)
    
    return files, start_date, end_date

def get_square_boundary(grid):
    b_ind = set()
    columns = grid.nx
    rows = grid.ny
    for edge in [[0, columns], [rows-1, columns],
                 [rows, 0], [rows, columns-1]]:
        b = np.array([[edge[0]]*edge[1], list(range(edge[1]))])
        b = b.transpose().tolist()
        b = set([tuple(b[i]) for i in range(edge[1])])
        b_ind = b_ind.union(b)
    return b_ind

def get_circular_boundary(grid):
    radius = grid.x['data'][-1]
    # Assume a regular grid
    dx = grid.x['data'][1] - grid.x['data'][0]
    offset = np.sqrt(2*dx**2)
    X, Y = np.meshgrid(grid.x['data'], grid.y['data'], indexing='ij')
    radii = np.sqrt(X**2+Y**2)
    
    b_ind = np.argwhere(np.logical_and(radii >= radius, 
                                       radii < radius + dx))
    b_ind_set = set([tuple(b_ind[i]) for i in range(b_ind.shape[0])])
        
    plt.pcolor(np.logical_and(radii >= radius-offset/2, 
                              radii < radius+offset/2))
    return b_ind_set

In [3]:
with open('/g/data/w40/esh563/CPOL_analysis/TINT_tracks/tracks_obj_20052006.pkl', 'rb') as f:
    tracks_obj = pickle.load(f)

In [10]:
filenames = CPOL_files_from_datetime_list(np.arange(np.datetime64('2005-12-01'), 
                                      np.datetime64('2006-04-01 00:00'), 
                                      np.timedelta64(10, 'm')))[0]
# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity')
         for fn in filenames)

with open('/g/data/w40/esh563/CPOL_analysis/TINT_tracks/circ_b_ind_set.pkl', 'rb') as f:
    b_ind_set = pickle.load(f)

# Define settings for tracking
settings = {
    'MIN_SIZE' : [1, 1, 500, 500],
    'FIELD_THRESH' : [40, 20, 15, 15],
    'ISO_THRESH' : [5, 5, 5, 5],
    'GS_ALT' : 1500,
    'SEARCH_MARGIN' : 10000,
    'FLOW_MARGIN' : 40000,
    'LEVELS' : np.array(
        [[0, 2500], 
         [2500, 5000],
         [5000, 7500],
         [7500, 10000]]
    ),
    'TRACK_INTERVAL' : 0,
    'BOUNDARY_GRID_CELLS' : b_ind_set
}

# Calculate high and low level tracks
tracks_obj  = tint.Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS', 
                  'TRACK_INTERVAL', 'ISO_THRESH', 'SEARCH_MARGIN',
                  'FLOW_MARGIN', 'BOUNDARY_GRID_CELLS'
                 ]:
    tracks_obj.params[parameter] = settings[parameter]

# Calculate tracks
tracks_obj.get_tracks(grids)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "") 

out_file_name = ('/g/data/w40/esh563/CPOL_analysis/TINT_tracks/'
                 + 'tracks_obj_{}.pkl'.format(dt))

with open(out_file_name, 'wb') as f:
    pickle.dump(tracks_obj, f)
        
# if __name__ == '__main__':
#     try:
#         test()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

Gathering files.
Calculating additional tracks properties.
Calculating system tracks.


Time elapsed: 70.0 minutes


In [88]:
lon_min = 131.0 - 1.6
lon_max = 131.0 + 1.6

lat_max = -12.2 + 1.6
lat_min = -12.2 - 1.6

# [filenames, start_time, end_time] = CPOL_files_from_datetime_list(np.arange(np.datetime64('2006-02-10 09:00'), 
#                                                                   np.datetime64('2006-02-10 16:00'), 
#                                                                   np.timedelta64(10, 'm')))

filenames, start_time, end_time = CPOL_files_from_TINT_obj(tracks_obj, '3171')

# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity')
         for fn in filenames)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_anim'

animate(tracks_obj, grids, out_path,
        start_datetime = start_time,
        end_datetime = end_time,
        lat_lines=np.arange(lat_min, lat_max, .2),
        lon_lines=np.arange(lon_min, lon_max, .2),
        tracers=True, keep_frames=False, dpi=100, alt=7500)

Gathering files.
Animating from 2006-03-28T02:50:09.000000000 to 2006-03-28T08:00:09.000000000.
Plotting scan at 2006-03-28T08:00:09.
Creating GIF - may take a few minutes.


In [91]:
# Let's filter by tilt direction and magnitude, velocity magnitude
# and significant area.
forward_tilt = ((-45 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= 45))
backward_tilt = ((-45 >= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                | (tracks_obj.system_tracks['sys_rel_tilt_dir'] >= 45))
left_tilt = ((45 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= 135))
right_tilt = ((-135 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= -45))
sig_tilt_mag = (tracks_obj.system_tracks['tilt_mag'] >= 2000)
vel_mag = np.sqrt(tracks_obj.system_tracks['u']**2 
                  + tracks_obj.system_tracks['v']**2)
sig_vel_mag = ((vel_mag >= 0) & (vel_mag <= 50))
# Note for CPOL 2.5 km, total scan area is only 66052 km^2. This makes 
# traditional MCS definitions of area > 30000 km^2 difficult to apply, 
# and still coherently calculate tilt. 
sig_area = ((tracks_obj.system_tracks['proj_area'] >= 1000) & 
            (tracks_obj.system_tracks['proj_area'] <= 40000))
not_border = (tracks_obj.system_tracks['touch_border']*6.25 / tracks_obj.system_tracks['proj_area']) < 0.01 

In [92]:
# Let's find the cells that were tracked for the most frames.
tracks_obj.system_tracks[sig_area & not_border & sig_vel_mag].groupby(level='uid').size().sort_values(ascending=False)[:20]

uid
1309    32
381     30
1757    28
1981    27
437     27
3299    27
2005    26
2718    26
2071    26
2503    26
2321    26
3171    26
2822    25
2109    25
2127    25
97      25
69      25
3226    24
121     23
796     23
dtype: int64

In [87]:
tracks_obj.system_tracks.xs('3171', level='uid')[['touch_border', 'proj_area', 'sys_rel_tilt_dir']]

,,touch_border,proj_area,sys_rel_tilt_dir
scan,time,,,
16755,2006-03-28 02:50:09,0,6262.50,NaN
16756,2006-03-28 03:00:09,4,7450.00,-174.163
16757,2006-03-28 03:10:12,7,7325.00,22.172
16758,2006-03-28 03:20:09,5,7237.50,-138.231
16759,2006-03-28 03:30:09,4,9112.50,52.133
16760,2006-03-28 03:40:09,0,9712.50,68.129
16761,2006-03-28 03:50:09,0,11012.50,-175.228
16762,2006-03-28 04:00:09,0,10662.50,-116.996
16763,2006-03-28 04:10:12,0,12581.25,150.246


In [86]:
uid = '2321'

filenames, start_time, end_time = CPOL_files_from_TINT_obj(tracks_obj, uid)

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_lagrangian'
animate(tracks_obj, grids, out_path, style='lagrangian', uid=uid, 
        alt_low=1500, alt_high=8000, keep_frames=False, dpi=100)

Gathering files.
Animating 31 frames
Plotting frame at 2006-03-06T19:30:08
Creating GIF - may take a few minutes.
